<a href="https://colab.research.google.com/github/periclesrialto/IndustrialAutomation/blob/master/AlarmMatrix_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math
import sys

In [ ]:
#Script configs
#(xlsx or csv)
file_format = 'xlsx'
#(word or dword)
var_type = 'dword'
#################################################################
if var_type == 'dword':
  var_base_len = 32
elif var_type == 'word':
  var_base_len = 16
else:
  print('Invalid Type')
#################################################################
if file_format == 'xlsx':
  #reads .xlsx format
  df = pd.read_excel('Alarm_matrix.xlsx', header = None)
elif file_format == 'csv':
  #reads .csv format
  df = pd.read_csv('Alarm_matrix.csv', header = None, sep=';')
else:
  print('Invalid Format')

In [ ]:
#Gather all the alarm categories on the table
alarm_category = []
for column in range(len(df.columns)):
  if df[column][1] == 'ALL':
    shift = 1
    while df[column + shift][1] != 'ALL':
      alarm_category.append(df[column+ shift][1])
      shift += 1
    break
for category in range(len(alarm_category)):
  alarm_category[category] = alarm_category[category].lower()
  alarm_category[category] = alarm_category[category].capitalize()
  alarm_category[category] += 'Group'

#Gather all the machine states on the table
machine_state = []
for column in range(len(df.columns)):
  if df[column][1] == 'ALL':
    machine_state.append(df[column][0])
for state in range(len(machine_state)):
  machine_state[state] = machine_state[state].replace(" ", "")

#Combines states and categories to create names for dataframe columns
column_names = []
for state in range(len(machine_state)):
  for category in range(len(alarm_category)):
    column_names.append(machine_state[state] + '.' + alarm_category[category])

#Create an empty dataframe using the previous generated names for the columns
#Insert the right amount of empty rows on the dataframe
dwords_df = pd.DataFrame(columns=column_names)
for empty in range(math.ceil(len(df.index)/var_base_len)):
  dwords_df = dwords_df.append(pd.Series(dtype='object'), ignore_index=True)
dwords_df

,Startup.EmergencyGroup,Startup.EndGroup,Startup.InterruptGroup,DryingUnder180.EmergencyGroup,DryingUnder180.EndGroup,DryingUnder180.InterruptGroup,DryingOver180.EmergencyGroup,DryingOver180.EndGroup,DryingOver180.InterruptGroup,Roasting.EmergencyGroup,Roasting.EndGroup,Roasting.InterruptGroup,Cooling.EmergencyGroup,Cooling.EndGroup,Cooling.InterruptGroup,Transport.EmergencyGroup,Transport.EndGroup,Transport.InterruptGroup
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Creates an empty list of strings for
dwords_range = (math.ceil(len(df.index)/var_base_len))*len(alarm_category)*len(machine_state)
dwords_list = ["" for x in range(dwords_range)]

#Extracts all the bits from the imported table and combine them
pointer = 0
row = 0
for column in range(len(df.columns)):
  if df[column][1] == 'ALL':
    for shift in range(1,4):
      for index in range(2,len(df.index)):
        dwords_list[pointer] += str(df[column+shift][index])
        row += 1
        if row==var_base_len and pointer < len(dwords_list):
          pointer += 1
          row = 0
        if index == (len(df.index)-1):
          for complete in range(var_base_len-row):
            dwords_list[pointer] += str(0)
          if pointer < len(dwords_list):
            pointer += 1
            row = 0
for invert in range(len(dwords_list)):
  dwords_list[invert] += '#2'
  dwords_list[invert] = dwords_list[invert][::-1]
dwords_list

['2#00000000000000000000000000000001',
 '2#00000000000000000000000000000000',
 '2#00000000000000000000000000000000',
 '2#00000000000000000000000000000000',
 '2#00000000000000000000000000000010',
 '2#00000000000000000000000000000000',
 '2#00000000000000000000000000000000',
 '2#00000000000000000000000000000000',
 '2#10101010101010101010101010101000',
 '2#10101010101010101010101010101010',
 '2#10101010101010101010101010101010',
 '2#00000000000000000000000000000000',
 '2#00000000000000000000000000000001',
 '2#00000000000000000000000000000000',
 '2#00000000000000000000000000000000',
 '2#00000000000000000000000000000000',
 '2#01010101010101010101010101010100',
 '2#01010101010101010101010101010101',
 '2#01010101010101010101010101010101',
 '2#00000000000000000000000000000001',
 '2#10101010101010101010101010101000',
 '2#10101010101010101010101010101010',
 '2#10101010101010101010101010101010',
 '2#00000000000000000000000000000000',
 '2#01010101010101010101010101010100',
 '2#010101010101010101010

In [ ]:
pointer = 0
for column in range(len(dwords_df.columns)):
  for index in range(len(dwords_df.index)):
    dwords_df.at[index,column_names[column]]=dwords_list[pointer]
    pointer += 1
dwords_df

,Startup.EmergencyGroup,Startup.EndGroup,Startup.InterruptGroup,DryingUnder180.EmergencyGroup,DryingUnder180.EndGroup,DryingUnder180.InterruptGroup,DryingOver180.EmergencyGroup,DryingOver180.EndGroup,DryingOver180.InterruptGroup,Roasting.EmergencyGroup,Roasting.EndGroup,Roasting.InterruptGroup,Cooling.EmergencyGroup,Cooling.EndGroup,Cooling.InterruptGroup,Transport.EmergencyGroup,Transport.EndGroup,Transport.InterruptGroup
0,2#00000000000000000000000000000001,2#00000000000000000000000000000010,2#10101010101010101010101010101000,2#00000000000000000000000000000001,2#01010101010101010101010101010100,2#10101010101010101010101010101000,2#01010101010101010101010101010100,2#00000000000000000000000000000001,2#00000000000000000000000000000010,2#00000000000000000000000000000010,2#10101010101010101010101010101000,2#00000000000000000000000000000001,2#00000000000000000000000000000001,2#10101010101010101010101010101000,2#01010101010101010101010101010100,2#10101010101010101010101010101000,2#00000000000000000000000000000010,2#01010101010101010101010101010100
1,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#10101010101010101010101010101010,2#00000000000000000000000000000000,2#01010101010101010101010101010101,2#10101010101010101010101010101010,2#01010101010101010101010101010101,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#10101010101010101010101010101010,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#10101010101010101010101010101010,2#01010101010101010101010101010101,2#10101010101010101010101010101010,2#00000000000000000000000000000000,2#01010101010101010101010101010101
2,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#10101010101010101010101010101010,2#00000000000000000000000000000000,2#01010101010101010101010101010101,2#10101010101010101010101010101010,2#01010101010101010101010101010101,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#10101010101010101010101010101010,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#10101010101010101010101010101010,2#01010101010101010101010101010101,2#10101010101010101010101010101010,2#00000000000000000000000000000000,2#01010101010101010101010101010101
3,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000001,2#00000000000000000000000000000000,2#00000000000000000000000000000001,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000001,2#00000000000000000000000000000000,2#00000000000000000000000000000000,2#00000000000000000000000000000001


In [ ]:
#Combines all the previous data and create the code used on the PLC

print('(*')
print('!!!AVISO!!!')
print('Este código foi gerado automaticamente')
print('Para que o script em SCL funcione:\n')

print('Crie um array de alarmes:')
print(' --> DB_ALM_AREA como Array[0..'+str(len(dwords_df.index)-1)+'] of '+var_type+'\n')

print('Crie um datatype que contenha:')
for category in range(len(alarm_category)):
  print(' --> '+str(alarm_category[category])+' como '+var_type)
  print(' --> '+str(alarm_category[category]).replace('Group','Out')+' como Bool')

print('\nCrie instâncias do datatype com os nomes:')
for state in range(len(machine_state)):
    print(' --> '+str(machine_state[state]))

print('\nCopie o script abaixo para o TIA Portal:\n')
print('*)\n\n')
category_pointer = 0
for column in range(len(dwords_df.columns)): 
  if category_pointer == 0:
    print('REGION Masks '+str(dwords_df.columns[column].split('.')[0])+'\n')
  print(' REGION Comparing Alarm Matrix with '+str(dwords_df.columns[column])+'\n')
  for alarm_dw in range(len(dwords_df.index)):
    print('  IF (DB_ALM_AREA['+str(alarm_dw)+'] AND '+str(dwords_df[dwords_df.columns[column]][alarm_dw])+') <> 0 THEN')
    print('   '+str(".".join(dwords_df.columns[column].split('.')))+'.%X'+str(alarm_dw)+' := TRUE;')
    print('  ELSE')
    print('   '+str(".".join(dwords_df.columns[column].split('.')))+'.%X'+str(alarm_dw)+' := FALSE;')
    print('  END_IF;\n')
    category_pointer += 1
  print('  REGION Compiles the '+str(" ".join(dwords_df.columns[column].split('.')))+' mask output\n')
  print('   IF '+str(".".join(dwords_df.columns[column].split('.')))+' <> 0 THEN')
  print('     '+str(dwords_df.columns[column]).replace('Group','Out')+' := TRUE;')
  print('   ELSE')
  print('     '+str(dwords_df.columns[column]).replace('Group','Out')+' := FALSE;')
  print('   END_IF;\n')
  print('  END_REGION\n')
  print(' END_REGION\n')
  
  if category_pointer == (len(dwords_df.index)*len(alarm_category)):
    print('END_REGION\n')
    category_pointer = 0

(*
!!!AVISO!!!
Este código foi gerado automaticamente
Para que o script em SCL funcione:

Crie um array de alarmes:
 --> DB_ALM_AREA como Array[0..3] of dword

Crie um datatype que contenha:
 --> EmergencyGroup como dword
 --> EmergencyOut como Bool
 --> EndGroup como dword
 --> EndOut como Bool
 --> InterruptGroup como dword
 --> InterruptOut como Bool

Crie instâncias do datatype com os nomes:
 --> Startup
 --> DryingUnder180
 --> DryingOver180
 --> Roasting
 --> Cooling
 --> Transport

Copie o script abaixo para o TIA Portal:

*)


REGION Masks Startup

 REGION Comparing Alarm Matrix with Startup.EmergencyGroup

  IF (DB_ALM_AREA[0] AND 2#00000000000000000000000000000001) <> 0 THEN
   Startup.EmergencyGroup.%X0 := TRUE;
  ELSE
   Startup.EmergencyGroup.%X0 := FALSE;
  END_IF;

  IF (DB_ALM_AREA[1] AND 2#00000000000000000000000000000000) <> 0 THEN
   Startup.EmergencyGroup.%X1 := TRUE;
  ELSE
   Startup.EmergencyGroup.%X1 := FALSE;
  END_IF;

  IF (DB_ALM_AREA[2] AND 2#00000000000000